In [2]:
import numpy as np
from numpy import save
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from sklearn import preprocessing
from keras.optimizers import Adam, SGD
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten,Dropout,MaxPooling2D,BatchNormalization,AveragePooling2D
import matplotlib.pyplot as plt

# Load dataset
# Change TAG value to load different dataset
# TAG == 0 MFCCS data
# TAG == 1 SPECTROGAM data
TAG = 0 # or 1

if TAG == 0:
    ds_spectro = np.load("ds_mfccs.npy")
if TAG == 1:
    ds_spectro = np.load("ds_spectro.npy")

labels = np.load("labels.npy")

# shuffle trainset before splitting
randomize = np.arange(len(ds_spectro))
np.random.shuffle(randomize)
ds_spectro = ds_spectro[randomize]
labels = labels[randomize]

# split trainset 70-30
X_train_flat, X_test_flat, y_train_flat, y_test_flat = train_test_split( ds_spectro, labels, test_size = 0.33, random_state = 42)
print("TRAIN: ", X_train_flat.shape, ' ', y_train_flat.shape)
print("TEST: ",X_test_flat.shape, ' ', y_test_flat.shape)


tot_train_examples = X_train_flat.shape[0]
tot_test_examples = X_test_flat.shape[0]

if TAG == 0:
    width=40
    height=188
    
if TAG == 1:
    width = 129
    height = 428

channels = 1 # features channel
f_size = 32 # filters

Xtrain_reshaped = X_train_flat.reshape(tot_train_examples,width,height,channels)
Xtest_reshaped = X_test_flat.reshape(tot_test_examples,width,height,channels)

# LABEL array cast as cathegories
le = preprocessing.LabelEncoder()
le.fit(y_train_flat)
y_train_cat = le.transform(y_train_flat)
y_test_cat = le.transform(y_test_flat)
y_train_cat = to_categorical(y_train_cat, num_classes=2, dtype="float32")
y_test_cat = to_categorical(y_test_cat, num_classes=2, dtype="float32")

# CNN model
model = Sequential()

model.add(Conv2D(f_size, kernel_size=3, strides=(2, 2), activation='relu', input_shape=(width,height,channels)))
model.add(MaxPooling2D())
model.add(BatchNormalization())

f_size+=32
model.add(Conv2D(f_size, kernel_size=3, activation='relu',padding='same'))
model.add(MaxPooling2D())
model.add(BatchNormalization())

f_size-=48
model.add(Conv2D(f_size, kernel_size=3, activation='relu',padding='same'))
model.add(MaxPooling2D())
model.add(BatchNormalization())

model.add(Flatten())
model.add(Dense(2, activation='softmax'))

# Optimizer
epochs = 50
batch_size = 8
learning_rate = 0.01

opt = Adam(learning_rate=learning_rate,beta_1=0.9,beta_2=0.999,epsilon=1e-07,amsgrad=False)
#opt = SGD(learning_rate=learning_rate, momentum=0.1, nesterov=False)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

#TRAIN
history = model.fit(Xtrain_reshaped, y_train_cat, validation_data=(Xtest_reshaped, y_test_cat), epochs=epochs,batch_size=batch_size, shuffle=True)

#EVALUATION
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()


(894, 40, 188)   (894,)
(441, 40, 188)   (441,)
New shape:  (40, 188, 1)
['F' 'M']
(894, 2)    (441, 2)
['F' 'F' 'M' 'F' 'M' 'F' 'F' 'M' 'M']    [[1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]]
Train on 894 samples, validate on 441 samples
Epoch 1/50
894/894 [==============================] - 2s 3ms/step - loss: 0.3635 - accuracy: 0.8971 - val_loss: 0.4478 - val_accuracy: 0.8594
Epoch 2/50
894/894 [==============================] - 2s 2ms/step - loss: 0.1869 - accuracy: 0.9407 - val_loss: 0.2005 - val_accuracy: 0.9252
Epoch 3/50
824/894 [==========================>...] - ETA: 0s - loss: 0.1026 - accuracy: 0.9672

KeyboardInterrupt: 